In [2]:
import numpy as np
from collections import defaultdict
import builtins
import matplotlib.pyplot as plt

In [3]:
def sigmoid(s):
    # TODO: Implement
    # You will find this function useful.
    return np.exp(s)/(1+np.exp(s))


def normalized_gradient(X, Y, beta, l):
    """
    :param X: data matrix (2 dimensional np.array)
    :param Y: response variables (1 dimensional np.array)
    :param beta: value of beta (1 dimensional np.array)
    :param l: regularization parameter lambda
    :return: normalized gradient, i.e. gradient normalized according to data
    """
    # TODO: Implement
    beta=beta.reshape(beta.shape[0],1)
    grad=np.ones(X.shape[1])
    for i in range(X.shape[1]):
        sum=0
        for j in range(X.shape[0]):
            sum+=X[j,i]*Y[j]*(1-sigmoid(Y[j]*beta.T.dot(X[j,:])))

        grad[i]=2*l*beta[i]-sum

    grad/=X.shape[0]

    return grad


def gradient_descent(X, Y, epsilon=1e-6, l=1, step_size=1e-4, max_steps=1000):
    """
    Implement gradient descent using full value of the gradient.
    :param X: data matrix (2 dimensional np.array)
    :param Y: response variables (1 dimensional np.array)
    :param l: regularization parameter lambda
    :param epsilon: approximation strength
    :param max_steps: maximum number of iterations before algorithm will
        terminate.
    :return: value of beta (1 dimensional np.array)
    """
    X_norm=np.copy(X)
    X_mean = np.mean(X,axis=0)
    X_std=np.std(X,axis=0)
    #print(X_std)
    for i in range(1,X.shape[1]):
        X_norm[:,i]=(X_norm[:,i]-X_mean[i])/X_std[i]
    X_var=np.var(X,axis=0)

    beta = np.ones(X.shape[1])
    for s in range(max_steps):
        #if s % 1000 == 0:
            #print(s, beta)
        # TODO: Implement iterations.
        grad_beta = normalized_gradient(X_norm, Y, beta, l)
        if np.linalg.norm(step_size * grad_beta) < epsilon:

            break
        else:
            beta = beta - step_size * grad_beta
        pass
    beta[0] = beta[0] - sum(X_mean[j] * beta[j] / X_std[j] for j in range(1, X.shape[1]))
    for i in range(1, X.shape[1]):
        beta[i] = beta[i] / X_std[i]
    return beta


def lr_predict(X, beta):
    """
    :param X: 2 dimensional python list or numpy 2 dimensional array representing the data to be predicted
    :param beta: 1 dimentional python list or numpy array representing the weights returned by our model 
        (in this case logistic regression)
    
    :return: 1 dimentional python list or numpy array, the predicted labels y_i for correspinding X_i
    """
    teta = [sigmoid(beta.T.dot(x)) for x in X]
    y = [1 if t > 0.5 else 0 for t in teta]
    return y

In [4]:
class DecisionNode(object):
    """
    README
    DecisionNode is a building block for Decision Trees.
    DecisionNode is a python class representing a  node in our decision tree
    node = DecisionNode()  is a simple usecase for the class
    you can also initialize the class like this:
    node = DecisionNode(column = 3, value = "Car")
    In python, when you initialize a class like this, its __init__ method is called 
    with the given arguments. __init__() creates a new object of the class type, and initializes its 
    instance attributes/variables.
    In python the first argument of any method in a class is 'self'
    Self points to the object which it is called from and corresponds to 'this' from Java

    """

    def __init__(self,
                 column=None,
                 value=None,
                 false_branch=None,
                 true_branch=None,
                 current_results=None,
                 is_leaf=False,
                 result=None):
        """
        column is the column of the feature 
        value is the value of the feature of the specified column
        false_branch and true_branch are of type DecisionNode
        current_results is a dictionary that shows, for the current node,
            how many of each results it has (for example {"a":0, "b":5, "c":45})
        is_leaf is boolean and shows if node is a leaf
        result is the most popular answer from curren_results. (in the above example "c")
        """
        self.column = column
        self.value = value
        self.false_branch = false_branch
        self.true_branch = true_branch
        self.current_results = current_results
        self.is_leaf = is_leaf
        max = -1
        for i in current_results.keys():
            a = current_results[i]
            if a > max:
                max = a
                self.result = i

def dict_of_values(data):
    """
    param data: a 2D Python list representing the data. Last column of data is Y.
    return: returns a python dictionary showing how many times each value appears in Y

    for example 
    data = [[1,'yes'],[1,'no'],[1,'yes'],[1,'yes']]
    dict_of_values(data)
    should return {'yes' : 3, 'no' :1}
    """
    results = defaultdict(int)
    for row in data:
        r = row[len(row) - 1]
        results[r] += 1
    return dict(results)


def divide_data(data, feature_column, feature_val):
    """
    this function dakes the data and divides it in two parts by a line. A line
    is defined by the feature we are considering (feature_column) and the target 
    value. The function returns a tuple (data1, data2) which are the desired parts of the data.
    For int or float types of the value, data1 have all the data with values >= feature_val
    in the corresponding column and data2 should have rest.
    For string types, data1 should have all data with values == feature val and data2 should 
    have the rest.

    param data: a 2D Python list representing the data. Last column of data is Y.
    param feature_column: an integer index of the feature/column.
    param feature_val: can be int, float, or string
    return: a tuple of two 2D python lists
    """
    data1=[]
    data2=[]
    if type(feature_val) == int or type(feature_val) == float:
        for i in range(len(data)):
            if data[i][feature_column]>=feature_val:
                data1.append(data[i])
            else:
                data2.append(data[i])
    if type(feature_val) == str:
        for i in range(len(data)):
            if data[i][feature_column]==feature_val:
                data1.append(data[i])
            else:
                data2.append(data[i])
    return (data1, data2)


def gini_impurity(data1, data2):

    """
    Given two 2D lists of compute their gini_impurity index. 
    Remember that last column of the data lists is the Y
    Lets assume y1 is y of data1 and y2 is y of data2.
    gini_impurity shows how diverse the values in y1 and y2 are.
    gini impurity is given by 

    N1*sum(p_k1 * (1-p_k1)) + N2*sum(p_k2 * (1-p_k2))

    where N1 is number of points in data1
    p_k1 is fraction of points that have y value of k in data1
    same for N2 and p_k2


    param data1: A 2D python list
    param data2: A 2D python list
    return: a number - gini_impurity 
    """
    dict1,dict2=dict_of_values(data1),dict_of_values(data2)
    k1,k2=list(dict1.values()),list(dict2.values())
    N1,N2 = sum(k1),sum(k2)

    p_k1,p_k2=[x/N1 for x in k1],[x/N2 for x in k2]

    gini1=0
    gini2=0
    for i in range(len(k1)):
        gini1+= N1*(p_k1[i]*(1-p_k1[i]))
    for i in range(len(k2)):
        gini2+= N2*(p_k2[i]*(1-p_k2[i]))
    gini=gini1+gini2

    return gini


def build_tree(data, current_depth=0, max_depth=1e10):
    """
    build_tree is a recursive function.
    What it does in the general case is:
    1: find the best feature and value of the feature to divide the data into
    two parts
    2: divide data into two parts with best feature, say data1 and data2
        recursively call build_tree on data1 and data2. this should give as two 
        trees say t1 and t2. Then the resulting tree should be 
        DecisionNode(...... true_branch=t1, false_branch=t2) 


    In case all the points in the data have same Y we should not split any more, and return that node
    For this function we will give you some of the code so its not too hard for you ;)
    
    param data: param data: A 2D python list
    param current_depth: an integer. This is used if we want to limit the numbr of layers in the
        tree
    param max_depth: an integer - the maximal depth of the representing
    return: an object of class DecisionNode

    """
    if len(data) == 0:
        return DecisionNode(is_leaf=True)

    if(current_depth == max_depth):
        return DecisionNode(current_results=dict_of_values(data))

    if(len(dict_of_values(data)) == 1):
        return DecisionNode(current_results=dict_of_values(data), is_leaf=True)

    #This calculates gini number for the data before dividing 
    self_gini = gini_impurity(data, [])

    #Below are the attributes of the best division that you need to find. 
    #You need to update these when you find a division which is better
    best_gini = 1e10
    best_column = None
    best_value = None
    best_split = None

   
    def column(matrix, i):
        return [row[i] for row in matrix]

    for i in range(len(data[0])-1):
        for j in column(data,i):
            gini=gini_impurity(divide_data(data,i,j)[0],divide_data(data,i,j)[1])
            if gini<best_gini:
                best_gini=gini
                best_column=i
                best_value=j
                best_split=(divide_data(data,i,j)[0],divide_data(data,i,j)[1])
    #print(best_gini,best_column,best_value,best_split)
   
    #recursively call build tree, construct the correct return argument and return
    t1=build_tree(best_split[1])
    t2=build_tree(best_split[0])
    return DecisionNode(best_column,best_value,t1,t2, dict_of_values(data))
        


def print_tree(tree, indent=''):
    # Is this a leaf node?
    if tree.is_leaf:
        print(str(tree.current_results))
    else:
        # Print the criteria
        #         print (indent+'Current Results: ' + str(tree.current_results))
        print('Column ' + str(tree.column) + ' : ' + str(tree.value) + '? ')

        # Print the branches
        print(indent + 'True->', end="")
        print_tree(tree.true_branch, indent + '  ')
        print(indent + 'False->', end="")
        print_tree(tree.false_branch, indent + '  ')


In [26]:
class DecisionTree(object):
    """
    DecisionTree class, that represents one Decision Tree

    :param max_tree_depth: maximum depth for this tree.
    """
    def __init__(self, max_tree_depth):
        self.max_depth = max_tree_depth

    def fit(self, X, Y):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :param Y: 1 dimensional python list or numpy 1 dimensional array
        
        :return: a tree of self.max_tree_depth with helps to classify the data
        """
        
            
        if type(X) is np.ndarray:
            X=X.tolist()
        if type(Y) is np.ndarray:
            Y=Y.tolist()

        data=[X[i]+[Y[i]] for i in range(len(X))]
        
        self.root=build_tree(data,max_depth=self.max_depth)



    def predict_x(self,x, v):
        
        """
        :param x: 1 dimensional python list or numpy 2 dimensional array
        :param v: the corrent node of the tree
       
        :return: choose and recursively call on the next node based on the values of x
        """
        
        if v.is_leaf==True:
            return v.result
        else:
            xval=x[v.column]
            val=v.value
            if type(val) == int or type(val) == float:
                if xval>val:
                    return self.predict_x(x,v.true_branch)
                else:
                    return self.predict_x(x, v.false_branch)
            if type(val)==str:
                if xval==val:
                    return self.predict_x(x,v.true_branch)
                else:
                    return self.predict_x(x,v.false_branch)


    def predict(self, X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        calls predict1 and returns the final predicted labels for corresponding X's
        
        :return: Y - 1 dimension python list with labels
        """

        return [self.predict1(row,self.root) for row in X]

In [64]:
class RandomForest(object):
    """
    RandomForest a class, that represents Random Forests.

    :param num_trees: Number of trees in the random forest
    :param max_tree_depth: maximum depth for each of the trees in the forest.
    :param ratio_per_tree: ratio of points to use to train each of
        the trees.
    """
    def __init__(self, num_trees, max_tree_depth, ratio_per_tree=0.5):
        self.num_trees = num_trees
        self.max_tree_depth = max_tree_depth
        self.ratio_per_tree=ratio_per_tree
        self.trees = None
        
    def fit(self, X, Y):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :param Y: 1 dimensional python list or numpy 1 dimensional array
        
        :return: forest - collection of decision trees
        """
        self.trees = []
        np.random.seed(13)
        n = len(X)
        sz = int(n*self.ratio_per_tree)

        if type(X) is np.ndarray:
            X=X.tolist()
        if type(Y) is np.ndarray:
            Y=Y.tolist()
            
        for i in range(self.num_trees):
            idx = np.arange(n)
            np.random.shuffle(idx)
            X = [X[idx[i]] for i in range(n)]
            Y = [Y[idx[i]] for i in range(n)]
            X_train = [X[i] for i in range(sz)]
            Y_train = [Y[i] for i in range(sz)]
            tree = DecisionTree(self.max_tree_depth)
            tree.fit(X_train, Y_train)
            self.trees.append(tree)
        return self.trees 

    def predict(self, X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :return: (Y, conf), tuple with Y being 1 dimension python
        list with labels, and conf being 1 dimensional list with
        confidences for each of the labels.
        """

        def column(matrix, i):
            return [row[i] for row in matrix]
        def most_common(lst):
            return max(set(lst), key= lst.count)
        Y=[]
        predicted=[]
        answer=[]
        conf=[]
        for i in range(self.num_trees):
            Y.append(self.trees[i].predict(X))
        for i in range(len(Y[0])):
            predicted.append(column(Y,i))
            answer.append(most_common(predicted[i]))
            conf.append(predicted[i].count(answer[i])/len(predicted[i]))

        return (answer, conf)

In [65]:
def accuracy_score(Y_true, Y_predict):
    """
    :param Y_true: 2D python list or numpy array
    :param Y_predict: 2D python list or numpy array
    
    :return: integer that represents the accurasy - the percent of correctly predicted labels
    """
    accuracy=0
    for i in range(len(Y_true)):
        if Y_predict[i]==Y_true[i]:
            accuracy+=1
    return accuracy/len(Y_true)


def evaluate_performance():
    '''
    Evaluate the performance of decision trees and logistic regression,
    average over 1,000 trials of 10-fold cross validation

    Return:
      a matrix giving the performance that will contain the following entries:
      stats[0,0] = mean accuracy of decision tree
      stats[0,1] = std deviation of decision tree accuracy
      stats[1,0] = mean accuracy of logistic regression
      stats[1,1] = std deviation of logistic regression accuracy

    ** Note that your implementation must follow this API**
    '''

    # Load Data
    filename = 'P3/SPECTF.dat'
    data = np.loadtxt(filename, delimiter=',')
    X = data[:, 1:]
    y = np.array(data[:, 0])
    n, d = X.shape
    folds=10

    decision_tree_accuracies=[]
    random_forest_accuracies=[]
    log_regression_accuracies=[]

    for trial in range(3):
        # TODO: shuffle for each of the trials.
        # the following code is for reference only.
        idx = np.arange(n)
        np.random.seed(13)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]
        size=int(n-n/folds)

        print("trial", trial)

        # TODO: write your own code to split data (for cross validation)
        # the code here is for your reference.
        Xtrain = X[:size,:]# train on first 100 instances
        Xtest = X[size:,:]
        ytrain = y[:size]# test on remaining instances
        ytest = y[size:]

        # train the decision tree
        dt = DecisionTree(100)
        
        dt.fit(Xtrain, ytrain)

        # output predictions on the remaining data
        dt_pred = dt.predict(Xtest)
        dt_accuracy = accuracy_score(ytest, dt_pred)
        decision_tree_accuracies.append(dt_accuracy)

        #train random forest
        rf= RandomForest(10,100)
        rf.fit(Xtrain,ytrain)
        rf_pred= rf.predict(Xtest)[0]
        rf_accuracy= accuracy_score(ytest,rf_pred)
        random_forest_accuracies.append(rf_accuracy)

        #logistic regression
        lr_beta=gradient_descent(Xtrain,ytrain,step_size=1e-1,max_steps=100)

        lr_pred=lr_predict(Xtest,lr_beta)
        lr_accuracy=accuracy_score(ytest,lr_pred)
        log_regression_accuracies.append(lr_accuracy)






    # compute the training accuracy of the model
    meanDecisionTreeAccuracy = np.mean(decision_tree_accuracies)
    stddevDecisionTreeAccuracy = np.std(decision_tree_accuracies)
    meanLogisticRegressionAccuracy = np.mean(log_regression_accuracies)
    stddevLogisticRegressionAccuracy = np.std(log_regression_accuracies)
    meanRandomForestAccuracy = np.mean(random_forest_accuracies)
    stddevRandomForestAccuracy = np.std(random_forest_accuracies)

    # make certain that the return value matches the API specification
    stats = np.zeros((3, 2))
    stats[0, 0] = meanDecisionTreeAccuracy
    stats[0, 1] = stddevDecisionTreeAccuracy
    stats[1, 0] = meanRandomForestAccuracy
    stats[1, 1] = stddevRandomForestAccuracy
    stats[2, 0] = meanLogisticRegressionAccuracy
    stats[2, 1] = stddevLogisticRegressionAccuracy
    return stats



In [66]:
stats = evaluate_performance()
print ("Decision Tree Accuracy = ", stats[0, 0], " (", stats[0, 1], ")")
print ("Random Forest Tree Accuracy = ", stats[1, 0], " (", stats[1, 1], ")")
print ("Logistic Reg. Accuracy = ", stats[2, 0], " (", stats[2, 1], ")" )

trial 0
trial 1
trial 2
Decision Tree Accuracy =  0.728395061728  ( 0.0461933010713 )
Random Forest Tree Accuracy =  0.79012345679  ( 0.0761038765799 )
Logistic Reg. Accuracy =  0.197530864198  ( 0.0872971334798 )
